In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import pickle
import os

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
cifar_dir = os.path.expanduser("~/JPTR_NTBK/CIFAR_10_DATA/cifar-10-batches-py/")

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

def load_cifar_data():
    X_train, y_train = [], []

    for i in range(1, 6):
        data_batch = unpickle(os.path.join(cifar_dir, f"data_batch_{i}"))
        X_train.append(data_batch[b'data'])
        y_train.append(data_batch[b'labels'])

    X_train = np.concatenate(X_train, axis=0).reshape(-1, 32, 32, 3)
    y_train = np.concatenate(y_train, axis=0)

    test_batch = unpickle(os.path.join(cifar_dir, "test_batch"))
    X_test = test_batch[b'data'].reshape(-1, 32, 32, 3)
    y_test = np.array(test_batch[b'labels'])

    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_cifar_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
y = tf.placeholder(tf.int32, [None], name="y")

conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu)(X)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(conv1)

conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu)(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(conv2)

conv3 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation=tf.nn.relu)(pool2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(conv3)

flat = tf.keras.layers.Flatten()(pool3)

fc1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flat)
fc2 = tf.keras.layers.Dense(units=128, activation=tf.nn.relu)(fc1)

logits = tf.keras.layers.Dense(units=10)(fc2)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(loss)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.cast(y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
init = tf.global_variables_initializer()
num_epochs = 5000
batch_size = 64

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(num_epochs):
        indices = np.random.choice(len(X_train), batch_size)
        X_batch, y_batch = X_train[indices], y_train[indices]

        loss_value, _ = sess.run([loss, optimizer], feed_dict={X: X_batch, y: y_batch})

        if epoch % 100 == 0:
            acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
            print(f"Epoch {epoch}: Loss = {loss_value:.4f}, Test Accuracy = {acc:.4%}")

    print("Training complete!")

    final_acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print(f"Final Test Accuracy: {final_acc:.4%}")



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NUM_LOCKS_IN_BLOCK=1

Epoch 0: Loss = 2.2957, Test Accuracy = 9.6200%
Epoch 100: Loss = 1.8067, Test Accuracy = 27.1900%
Epoch 200: Loss = 1.8518, Test Accuracy = 36.5100%
Epoch 300: Loss = 1.9202, Test Accuracy = 42.2300%
Epoch 400: Loss = 1.5711, Test Accuracy = 42.2300%
Epoch 500: Loss = 1.4919, Test Accuracy = 46.2900%
Epoch 600: Loss = 1.3083, Test Accuracy = 46.3500%
Epoch 700: Loss = 1.4640, Test Accuracy = 47.7400%
Epoch 800: Loss = 1.4064, Test Accuracy = 50.0600%
Epoch 900: Loss = 1.4172, Test Accuracy = 49.8500%
Epoch 1000: Loss = 1.3697, Test Accuracy = 54.0600%
Epoch 1100: Loss = 1.3006, Test Accuracy = 52.6700%
Epoch 1200: Loss = 1.4288, Test Accuracy = 55.3500%
Epoch 1300: Loss = 1.2518, Test Accuracy = 55.5400%
Epoch 1400: Loss = 1.1168, Test Accuracy = 55.1800%
Epoch 1500: Loss = 1.1808, Test Accuracy = 55.7500%
Epoch 1600: Loss = 1.0110, Test Accuracy = 56.5600%
Epoch 1700: Loss = 1.1468, Test Accuracy = 58.0500%
Epoch 1800: Loss = 1.1177, Test Accuracy = 56.1000%
Epoch 1900: Loss = 1.1259